In [2]:
import pandas 
import psycopg2
import datetime
import time
import os
conn = psycopg2.connect(host = '127.0.0.1' , dbname = 'postgres' , user = 'postgres' , password = '')
cur = conn.cursor()

# 【 Aggregate 】- All Investor

In [1]:
def INSTI_INVESTOR_SUMMARIZE(work_D,cursor):
    
    cur = cursor
    
    for D in work_D:
        
        process_day = '{:0>4}-{:0>2}-{:0>2}'.format(D.year , D.month , D.day)
        
        cur.execute("delete from ods.institutional_investor where date = '{}'".foramt(process_day), con = conn)
        cur.execute("commit")
        
        ii = pandas.read_sql("select main.date , main.no , main.quantity , main.type , d.date_no from \
                         (select date , lpad(no,4,'0') as no , quantity , 'Foreign Investor' as type from foreign_investor where date = '{0}'\
                          union\
                          select date , lpad(no,4,'0') as no , quantity , 'Investment Trust' as type from investment_trust where date = '{0}'\
                          union\
                          select date , lpad(no,4,'0') as no , quantity , 'Dealer' as type from dealer where date = '{0}' ) main join work_date d on main.date = d.date".format(process_day), con = conn)
        
        for _ ,data in ii.iterrows():
            cur.execute("insert into ods.institutional_investor (date ,no , quantity , type , date_no) values (%s,%s,%s,%s,%s)",data)
        cur.execute('commit')
        print('【 Institional Investor 】{} data inserted .'.format(process_day))

# 【 Aggregate 】- Stock SMA

In [ ]:
def STOCK_SMA(work_D , cursor):
    insert_D = work_D
    cur = cursor
    
    for iD in insert_D:
        
        # 讀取完整STOCK完整DAILY資料
        main = pandas.read_sql("select main.*, avg.price as avg_p from (select stock.no , stock.name , stock.close , stock.quantity , stock.date , wk.date_no                              from work_date wk                              left join stock_daily stock                                on wk.date = stock.date ) main                      join stock_daily_avg_price avg on main.date = avg.date and main.no = avg.no ", con = conn)
        
        # 將欄位 no , name , date 處理成category格式減少記憶體負擔    
        main[['no','name']] = main[['no','name']].astype('category')
        main['date'] = main['date'].astype('string')
        
        # 新增每日總量欄位 = 均價 × 張數 
        main['amount'] = main.avg_p * main.quantity
        main.sort_values(['no','date_no'], inplace = True)
        
        # 新增前59天JOIN KEY
        for i in range(1,60):
            main['key_{}'.format(i)] = main['date_no'] - i
            main['key_{}'.format(i)] = main['key_{}'.format(i)].astype('category')
        
        # tmp為LEFT JOIN的表格 ，只留需計算的欄位
        tmp = main[['no','date_no','amount','quantity','close']]
        
        # 取得處理日期的DATE_NO , 並留下需insert的日期資料
        iD = '{:0>4}-{:0>2}-{:0>2}'.format(iD.year,iD.month,iD.day)
        ino = pandas.read_sql("select distinct date_no , date from work_date where date = '{}'".format(iD) , con = conn)['date_no'][0]
        main = main[main['date_no'] == ino]
        
        print('Start : \033[1mMERGE\033[0m data within 60 days')
        
        # 將60天內的資料合併成一列 , 並刪除join key
        for i in range(1,60):
            main = pandas.merge(main , tmp , left_on = ['no','key_{}'.format(i)] , right_on = ['no','date_no'] , suffixes = ('','_y_{}'.format(i)))
            main.drop(['date_no_y_{}'.format(i),'key_{}'.format(i)], axis = 1 , inplace = True)
    
        print('Start : \033[1mAGGREGATE\033[0m close & average price')   
    
    # 計算各頻率 SMA & AVG
        freq = [5,10,20,60]
        for f in freq:
            main['sma_{}'.format(f)] = round(main.filter(like = 'close').iloc[:,:f].mean(axis =1),2)
            main['amt_{}'.format(f)] = main.filter(like = 'amount').iloc[:,:f].sum(axis =1)
            main['qty_{}'.format(f)] = main.filter(like = 'quantity').iloc[:,:f].sum(axis =1)
            main['avg_{}'.format(f)] = round((main['amt_{}'.format(f)] / main['qty_{}'.format(f)]),2)
            main.drop(['amt_{}'.format(f),'qty_{}'.format(f)] , axis = 1 , inplace = True)        
        main.drop(main.filter(regex=r'amount\_') , axis = 1 , inplace = True )
        main.drop(main.filter(regex=r'close\_') , axis = 1 , inplace = True )
        main.drop(main.filter(regex=r'quantity\_') , axis = 1 , inplace = True )
        main['date_no'] = pandas.read_sql("select date_no from work_date where date = '{}'".format(iD) , con = conn )['date_no'][0]
        main = main[['date','no','name','close','avg_p','avg_5','avg_10','avg_20','avg_60','sma_5','sma_10','sma_20','sma_60','date_no']]
        
        print('Start : \033[1mINSERT\033[0m to target table')
        
        if main.shape[0] == 0:
            pass
        else :
            display(main.head(1))
            for _ , data in main.iterrows():
                cur.execute("insert into ods.analyze_avg (date ,no ,name , close ,avg_p ,avg5 ,avg10 ,avg20 ,avg60 ,sma5 ,sma10 ,sma20 ,sma60 , date_no) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)" , data)
            cur.execute('commit')
        print('')

# 【 Aggregate 】- OTC SMA

In [ ]:
def OTC_SMA(work_D , cursor):
    insert_D = work_D
    cur = cursor
    
    for iD in insert_D:
        
        # 讀取完整STOCK完整DAILY資料
        main = pandas.read_sql("select main.*, avg.price as avg_p\
                     from (select stock.no , stock.name , stock.close , stock.quantity , stock.date , wk.date_no \
                             from work_date wk \
                             left join otc_daily stock \
                               on wk.date = stock.date ) main \
                     join stock_daily_avg_price avg on main.date = avg.date and main.no = avg.no ", con = conn)
        
        # 將欄位 no , name , date 處理成category格式減少記憶體負擔    
        main[['no','name']] = main[['no','name']].astype('category')
        main['date'] = main['date'].astype('string')
        
        # 新增每日總量欄位 = 均價 × 張數 
        main['amount'] = main.avg_p * main.quantity
        main.sort_values(['no','date_no'], inplace = True)
        
        # 新增前59天JOIN KEY
        for i in range(1,60):
            main['key_{}'.format(i)] = main['date_no'] - i
            main['key_{}'.format(i)] = main['key_{}'.format(i)].astype('category')
        
        # tmp為LEFT JOIN的表格 ，只留需計算的欄位
        tmp = main[['no','date_no','amount','quantity','close']]
        
        # 取得處理日期的DATE_NO , 並留下需insert的日期資料
        iD = '{:0>4}-{:0>2}-{:0>2}'.format(iD.year,iD.month,iD.day)
        ino = pandas.read_sql("select distinct date_no , date from work_date where date = '{}'".format(iD) , con = conn)['date_no'][0]
        main = main[main['date_no'] == ino]
        
        print('Start : \033[1mMERGE\033[0m data within 60 days')
        
        # 將60天內的資料合併成一列 , 並刪除join key
        for i in range(1,60):
            main = pandas.merge(main , tmp , left_on = ['no','key_{}'.format(i)] , right_on = ['no','date_no'] , suffixes = ('','_y_{}'.format(i)))
            main.drop(['date_no_y_{}'.format(i),'key_{}'.format(i)], axis = 1 , inplace = True)
        
        print('Start : \033[1mAGGREGATE\033[0m close & average price')   
        
        # 計算各頻率 SMA & AVG
        freq = [5,10,20,60]
        for f in freq:
            main['sma_{}'.format(f)] = round(main.filter(like = 'close').iloc[:,:f].mean(axis =1),2)
            main['amt_{}'.format(f)] = main.filter(like = 'amount').iloc[:,:f].sum(axis =1)
            main['qty_{}'.format(f)] = main.filter(like = 'quantity').iloc[:,:f].sum(axis =1)
            main['avg_{}'.format(f)] = round((main['amt_{}'.format(f)] / main['qty_{}'.format(f)]),2)
            main.drop(['amt_{}'.format(f),'qty_{}'.format(f)] , axis = 1 , inplace = True)
            
        # 刪除不需要INSERT的欄位
        main.drop(main.filter(regex=r'amount\_') , axis = 1 , inplace = True )
        main.drop(main.filter(regex=r'close\_') , axis = 1 , inplace = True )
        main.drop(main.filter(regex=r'quantity\_') , axis = 1 , inplace = True )
        main['date_no'] = pandas.read_sql("select date_no from work_date where date = '{}'".format(iD) , con = conn )['date_no'][0]
        main = main[['date','no','name','close','avg_p','avg_5','avg_10','avg_20','avg_60','sma_5','sma_10','sma_20','sma_60','date_no']]
        
        print('Start : \033[1mINSERT\033[0m to target table')
        
        if main.shape[0] == 0:
            pass
        else :
            display(main.head(1))
            for _ , data in main.iterrows():
                cur.execute("insert into ods.analyze_avg (date ,no ,name , close ,avg_p ,avg5 ,avg10 ,avg20 ,avg60 ,sma5 ,sma10 ,sma20 ,sma60 , date_no) \
                values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)" , data)
        
            cur.execute('commit')
        print('')
        

# 【 Aggregate 】- Foreign Investor OBS Summarize

In [ ]:
def FI_OBS_SUM(work_D , cursor):
    insert_D = work_D
    cur = cursor
    
    for iD in insert_D:

        #iD=datetime.datetime.today()
        D = '{:0>4}-{:0>2}-{:0>2}'.format(iD.year,iD.month,iD.day)
        
        # 取得所有股票代號 , 並分為五等份執行
        stock_no = pandas.read_sql("select no from stock_daily union select no from otc_daily",con = conn)
        step = int((len(stock_no)/5)+0.5)
        
        # 初始化批次區間變數
        x = 0
        y = step
        
        for part in range(5):      
            
            process_stock = tuple(stock_no.iloc[x:y,0].values)
            fi = pandas.read_sql("select distinct head.no , head.date , head.date_no , main.quantity \
                            from (select stock.* , wk.date_no \
                                    from (select no , date from stock_daily a where no in {0} union  select no , date from otc_daily b where no in {0}) stock \
                                    join work_date wk \
                                      on stock.date = wk.date) head \
                            left join (select distinct *  from foreign_investor ) main \
                              on head.no = main.no \
                             and head.date = main.date".format(process_stock) , con = conn).fillna(0)
            
            fi_tmp = fi[['no','date_no','quantity']]
            fi = fi[fi.date.astype('string') == D ]
        
            
            for i in range(1,60):
                fi['key_{}'.format(i)] = fi.date_no - i
            
            for i in range(1,60):
                fi = pandas.merge(fi,fi_tmp , left_on = ['no','key_{}'.format(i)], right_on =['no','date_no'] , suffixes = ('','_y{}'.format(i)))
        
                fi.drop(['key_{}'.format(i),'date_no_y{}'.format(i)], axis = 1 , inplace = True)
          
            fi['fi_sum5'] = fi.filter(like='quantity').iloc[:,:5].sum(axis = 1)
            fi['fi_sum10'] = fi.filter(like='quantity').iloc[:,:10].sum(axis = 1)
            fi['fi_sum20'] = fi.filter(like='quantity').iloc[:,:20].sum(axis = 1)
            fi['fi_sum60'] = fi.filter(like='quantity').iloc[:,:60].sum(axis = 1)
            fi.drop(fi.filter(regex = 'quantity\_').columns , axis =1 , inplace = True)
            fi['date_no'] = pandas.read_sql("select date_no from work_date where date = '{}'".format(D) , con = conn )['date_no'][0]
            fi = fi[['no','date','quantity','fi_sum5','fi_sum10','fi_sum20','fi_sum60','date_no']]
            display(fi.head(1))
        
            
            for _ , data in fi.iterrows():
                cur.execute('insert into ods.analyze_FI_OBS (no , date , quantity,sum5,sum10,sum20,sum60,date_no) \
            values (%s,%s,%s,%s,%s,%s,%s,%s)' , data)
            
            cur.execute('commit')
            print('【 Tech Analyze Foreign Investor Over Bought / Sold 】{0} data inserted {1}/5 .'.format(D,part+1))
            x = y
            y += step
        

# 【 Aggregate 】- Investment Trust OBS Summarize

In [ ]:
def IT_OBS_SUM(work_D , cursor):
    insert_D = work_D
    cur = cursor
    for iD in insert_D:
    
        #iD=datetime.datetime.today()
        D = '{:0>4}-{:0>2}-{:0>2}'.format(iD.year,iD.month,iD.day)
        
        # 取得所有股票代號 , 並分為五等份執行
        stock_no = pandas.read_sql("select no from stock_daily union select no from otc_daily",con = conn)
        step = int((len(stock_no)/5)+0.5)
        
        # 初始化批次區間變數
        x = 0
        y = step
        
        for part in range(5):      
            
            process_stock = tuple(stock_no.iloc[x:y,0].values)
            fi = pandas.read_sql("select distinct head.no , head.date , head.date_no , main.quantity \
                                from (select stock.* , wk.date_no \
                                        from (select no , date from stock_daily a where no in {0} union  select no , date from otc_daily b where no in {0}) stock \
                                        join work_date wk \
                                      on stock.date = wk.date) head \
                            left join (select distinct * from investment_trust) main \
                              on head.no = main.no \
                             and head.date = main.date".format(process_stock) , con = conn).fillna(0)
            
            fi_tmp = fi[['no','date_no','quantity']]
            
            fi = fi[fi.date.astype('string') == D ]
            
            
            for i in range(1,60):
                fi['key_{}'.format(i)] = fi.date_no - i
                
            for i in range(1,60):
                fi = pandas.merge(fi,fi_tmp , left_on = ['no','key_{}'.format(i)], right_on =['no','date_no'] , suffixes = ('','_y{}'.format(i)))
                
                fi.drop(['key_{}'.format(i),'date_no_y{}'.format(i)], axis = 1 , inplace = True)
            
            fi['it_sum5'] = fi.filter(like='quantity').iloc[:,:5].sum(axis = 1)
            fi['it_sum10'] = fi.filter(like='quantity').iloc[:,:10].sum(axis = 1)
            fi['it_sum20'] = fi.filter(like='quantity').iloc[:,:20].sum(axis = 1)
            fi['it_sum60'] = fi.filter(like='quantity').iloc[:,:60].sum(axis = 1)
            fi.drop(fi.filter(regex = 'quantity\_').columns , axis =1 , inplace = True)
            fi['date_no'] = pandas.read_sql("select date_no from work_date where date = '{}'".format(D) , con = conn )['date_no'][0]
            fi = fi[['no','date','quantity','it_sum5','it_sum10','it_sum20','it_sum60','date_no']]
            display(fi.head(1))
        
            
            for _ , data in fi.iterrows():
                cur.execute('insert into ods.analyze_IT_OBS (no , date , quantity,sum5,sum10,sum20,sum60,date_no) \
            values (%s,%s,%s,%s,%s,%s,%s,%s)' , data)
            
            cur.execute('commit')
            print('【 Tech Analyze Investment Trust Over Bought / Sold 】{0} data inserted {1}/5 .'.format(D,part+1))
            x = y
            y += step
        

# 【 Aggregate 】- Dealer OBS Summarize

In [ ]:
def DL_OBS_SUM(work_D,cursor):
    insert_D = work_D
    cur = cursor
    for iD in insert_D:
    
        #iD=datetime.datetime.today()
        D = '{:0>4}-{:0>2}-{:0>2}'.format(iD.year,iD.month,iD.day)
        
        # 取得所有股票代號 , 並分為五等份執行
        stock_no = pandas.read_sql("select no from stock_daily union select no from otc_daily",con = conn)
        step = int((len(stock_no)/5)+0.5)
        
        # 初始化批次區間變數
        x = 0
        y = step
        
        for part in range(5):      
            
            process_stock = tuple(stock_no.iloc[x:y,0].values)
            dl = pandas.read_sql("select distinct head.no , head.date , head.date_no , main.quantity \
                            from (select stock.* , wk.date_no \
                                    from (select no , date from stock_daily a where no in {0} union  select no , date from otc_daily b where no in {0}) stock \
                                    join work_date wk \
                                      on stock.date = wk.date) head \
                            left join (select distinct *  from dealer ) main \
                              on head.no = main.no \
                             and head.date = main.date".format(process_stock) , con = conn).fillna(0)
            
            dl_tmp = dl[['no','date_no','quantity']]
            dl = dl[dl.date.astype('string') == D ]
    
        
            for i in range(1,60):
                dl['key_{}'.format(i)] = dl.date_no - i
            
            for i in range(1,60):
                dl = pandas.merge(dl,dl_tmp , left_on = ['no','key_{}'.format(i)], right_on =['no','date_no'] , suffixes = ('','_y{}'.format(i)))
        
                dl.drop(['key_{}'.format(i),'date_no_y{}'.format(i)], axis = 1 , inplace = True)
          
            dl['dl_sum5'] = dl.filter(like='quantity').iloc[:,:5].sum(axis = 1)
            dl['dl_sum10'] = dl.filter(like='quantity').iloc[:,:10].sum(axis = 1)
            dl['dl_sum20'] = dl.filter(like='quantity').iloc[:,:20].sum(axis = 1)
            dl['dl_sum60'] = dl.filter(like='quantity').iloc[:,:60].sum(axis = 1)
            dl.drop(dl.filter(regex = 'quantity\_').columns , axis =1 , inplace = True)
            dl['date_no'] = pandas.read_sql("select date_no from work_date where date = '{}'".format(D) , con = conn )['date_no'][0]
            dl = dl[['no','date','quantity','dl_sum5','dl_sum10','dl_sum20','dl_sum60','date_no']]
            display(dl.head(1))
        
            
            for _ , data in dl.iterrows():
                cur.execute('insert into ods.analyze_DL_OBS (no , date , quantity,sum5,sum10,sum20,sum60,date_no) \
            values (%s,%s,%s,%s,%s,%s,%s,%s)' , data)
            
            cur.execute('commit')
            print('【 Tech Analyze Dealer Over Bought / Sold 】{0} data inserted {1}/5 .'.format(D,part+1))
            x = y
            y += step
    

# 【 Technology Analyse 】- MACD

In [4]:
def MACD(work_D , cursor , n = 12 , m = 26 , x = 9 ):
    n = 12
    m = 26
    x = 9
    
    for insert_D in work_D:

        D = '{:0>4}-{:0>2}-{:0>2}'.format(insert_D.year,insert_D.month,insert_D.day)
        all_stock = pandas.read_sql("select no from stock_daily where date = '{0}' union select no from otc_daily where date = '{0}'".format(D) , con = conn)['no']
        
        for i_stock in range(len(all_stock)):
            
            TA = all_stock[i_stock]
            today = pandas.read_sql("select m.* , d.date_no from (select date , no , close  from stock_daily where no = '{0}'  union select date , no , close  from otc_daily where no = '{0}' ) m \
                                                            join work_date d \
                                                              on m.date = d.date \
                                                           where d.date = '{1}' ".format(TA , D) , con = conn)
            today[['nEMA','mEMA','DIF','MACD','BAR']] = 0.0
            date_no = today['date_no'][0]
            yesterday = pandas.read_sql("select date , date_no , no , nEMA , mEMA , MACD from ods.macd where no = '{0}' and date_no = {1}".format(TA , date_no-1) , con = conn)
            
            if yesterday.shape[0] == 0:
                today['nEMA'] = today['close'] * 2 / (n + 1)
                today['mEMA'] = today['close'] * 2 / (m + 1)
                today['DIF'] = today['nEMA'] - today['mEMA']
                today['MACD'] = today['DIF'] * 2 / (x + 1)
                today['BAR'] = today['DIF'] - today['MACD']
        
            else:
                today['nEMA'] = ((yesterday['nema'][0] * (n - 1)) + (today['close'] * 2)) / (n + 1)
                today['mEMA'] = ((yesterday['mema'][0] * (m - 1)) + (today['close'] * 2)) / (m + 1)
                today['DIF'] = today['nEMA'] - today['mEMA']
                today['MACD'] = ((yesterday['macd'][0] * (x - 1)) + today['DIF'][0] * 2) / (x + 1)
                today['BAR'] = today['DIF'] - today['MACD']
            today[['BAR','nEMA','mEMA','MACD']] = round(today[['BAR','nEMA','mEMA','MACD']],3)     
            today = today[['date','date_no','no','BAR','nEMA','mEMA','MACD']]
        
            for _,data in today.iterrows():
                
                cur.execute("insert into ods.macd (date , date_no , no , macd_bar , nema , mema , macd) values (%s,%s,%s,%s,%s,%s,%s)",data)
            cur.execute("commit")
            
        print("【 MACD 】{} data inserted. ".format(insert_D))

# 【 Technology Analyse 】- CCT5

In [ ]:
def CCT5(work_D , cursor ):

    for D in work_D:
        
        insert_D = '{:0>4}-{:0>2}-{:0>2}'.format(D.year,D.month,D.day)
        
        # 取得股市中的5天內的日期
        html_D = pandas.read_sql("select y.date as first_day ,t.date as last_day \
                                    from work_date t \
                                    join work_date y \
                                      on t.date_no - 4 = y.date_no \
                                   where t.date = '{0}'".format(insert_D) , con = conn)
        # 排除週一到週五但無股市的日期
        if html_D.shape[0] != 0:
            
            first_D = str(html_D['first_day'][0])
            last_D = str(html_D['last_day'][0])
            
            # 讀取處理日期中的所有股票編號
            all_stock = pandas.read_sql("select main.no  from (select no from stock_daily where date = '{0}' union select no from otc_daily where date = '{0}') main left join (select no from ods.cct5 where date = '{0}') c on c.no = main.no where c.no is null and main.no != '1418'".format(insert_D),con = conn)
    
            for i in range(len(all_stock)):
                TA = all_stock['no'][i]
                
                # 讀取當日以前的交易量
                QTY = pandas.read_sql("select * from (select date , no , quantity from stock_daily where no = '{0}' and date <= '{1}' union select date , no , quantity from otc_daily where no = '{0}' and date <= '{1}') a order by date".format(TA,insert_D) , con = conn)
                # 判斷此股票是否有5天的交易日
                if len(QTY) >= 5:
                    
                    sum5 = QTY['quantity'].iloc[-5:].sum()
                    #html = 'http://sod.nsc.com.tw/z/zc/zco/zco.djhtm?a={0}&e={1}&f={2}'.format(TA,first_D,last_D)
                    html = 'https://fubon-ebrokerdj.fbs.com.tw/z/zc/zco/zco.djhtm?a={0}&e={1}&f={2}'.format(TA,first_D,last_D)
                    main = pandas.read_html(html)[2].iloc[-3,[1,6]]
                    
                    # 排除20日內無主力交易
                    if main[1] != '買超':
                    
                        cct5 = round((int(main[1]) - int(main[6]))/sum5,4)
                        date_no = pandas.read_sql("select date_no from work_date where date = '{}'".format(insert_D) , con = conn )['date_no'][0]
                        insert_data = pandas.DataFrame([[insert_D,TA,cct5,date_no]],columns = ['date','no','cct5','date_no'])
                        
                        for _ , data in insert_data.iterrows():
                            try:
                                cur.execute("insert into ods.cct5 (date , no , cct5,date_no) values (%s,%s,%s,%s)",data)
                            except :
                                cur.execute("insert into ods.cct5 (date , no , cct5,date_no) values (%s,%s,%s,%s)",(insert_D,TA,99.9999,date_no))
                        
                        cur.execute("commit")
                        if (i+1)%100 == 0:
                            print('{}  {:>5} / {:>5}'.format(insert_D , i+1,len(all_stock)))
                        elif (i+1)%len(all_stock) ==0:
                            print('{}  {:>5} / {:>5}'.format(insert_D , i+1,len(all_stock)))
                    else:
                        pass
                
                else :
                    pass
        else :
            pass


# 【 Technology Analyse 】- CCT20

In [ ]:
def CCT20(work_D , cursor ):

    for D in work_D:
        
        insert_D = '{:0>4}-{:0>2}-{:0>2}'.format(D.year,D.month,D.day)
        
        # 取得股市中的19天前的日期
        html_D = pandas.read_sql("select y.date as first_day ,t.date as last_day \
                                    from work_date t \
                                    join work_date y \
                                      on t.date_no - 19 = y.date_no \
                                   where t.date = '{0}'".format(insert_D) , con = conn)
        
        # 排除週一到週五但無股市的日期
        if html_D.shape[0] !=0:
            
            first_D = str(html_D['first_day'][0])
            last_D = str(html_D['last_day'][0])
            
            # 讀取處理日期中的所有股票編號
            all_stock = pandas.read_sql("select main.no  from (select no from stock_daily where date = '{0}' union select no from otc_daily where date = '{0}') main left join (select no from ods.cct20 where date = '{0}') c on c.no = main.no where c.no is null and main.no != '1418'".format(insert_D),con = conn)
    
            for i in range(len(all_stock)):
            
                TA = all_stock['no'][i]
                
                # 讀取當日以前的交易量
                QTY = pandas.read_sql("select * from (select date , no , quantity from stock_daily where no = '{0}' and date <= '{1}' union select date , no , quantity from otc_daily where no = '{0}' and date <= '{1}') a order by date".format(TA,insert_D) , con = conn)               
                
                # 判斷此股票是否有20天的交易日
                if len(QTY) >= 20:
                    
                    sum20 = QTY['quantity'].iloc[-20:].sum()
                    #html = 'http://sod.nsc.com.tw/z/zc/zco/zco.djhtm?a={0}&e={1}&f={2}'.format(TA,first_D,last_D)
                    html = 'https://fubon-ebrokerdj.fbs.com.tw/z/zc/zco/zco.djhtm?a={0}&e={1}&f={2}'.format(TA,first_D,last_D)
                    main = pandas.read_html(html)[2].iloc[-3,[1,6]]
                    
                    # 排除20日內無主力交易
                    if main[1] != '買超':
                    
                        cct20 = round((int(main[1]) - int(main[6]))/sum20,4)
                        date_no =pandas.read_sql("select date_no from work_date where date = '{}'".format(insert_D) , con = conn )['date_no'][0]
                        insert_data = pandas.DataFrame([[insert_D,TA,cct20,date_no]],columns = ['date','no','cct20','date_no'])
                        
                        for _ , data in insert_data.iterrows():
                            
                            try:
                                cur.execute("insert into ods.cct20 (date , no , cct20,date_no) values (%s,%s,%s,%s)",data)
                            except :
                                cur.execute("insert into ods.cct20 (date , no , cct20,date_no) values (%s,%s,%s,%s)",(insert_D,TA,99.9999,date_no))
                        cur.execute("commit")
                        if (i+1)%100 == 0:
                            print('{}  {:>5} / {:>5}'.format(insert_D , i+1,len(all_stock)))
                        elif (i+1)%len(all_stock) ==0 :
                            print('{}  {:>5} / {:>5}'.format(insert_D , i+1,len(all_stock)))
                                
                    else:
                        pass
                
                else :
                    pass
        else :
            pass 

# 【 Technology Analyse 】- KD

In [ ]:
def KD(work_D , cur):
    for D in work_D:
        insert_D = '{:0>4}-{:0>2}-{:0>2}'.format(D.year,D.month,D.day)
        all_list = pandas.read_sql("select main.no from (select date,no from stock_daily where date = '{0}'  union \
                         select date , no from otc_daily where date = '{0}' ) main left join ods.kd k on main.date =k.date and main.no = k.no where k.no is null".format(insert_D) , con = conn)['no'].tolist()
        
        date_list = tuple(pandas.read_sql("select cast(date as varchar) from work_date where date_no >= (select date_no from work_date where date = '{0}') -8 \
                                              and date_no <= (select date_no from work_date where date = '{0}')".format(insert_D) , con = conn)['date'].tolist())
        try :
            date_no = str(pandas.read_sql("select date_no from work_date where date = '{0}'".format(insert_D) , con = conn)['date_no'][0])
        except IndexError:
            pass
        
        for no in range(len(all_list)):
            kd_ini = pandas.read_sql("select * from ods.kd where no = '{0}'".format(all_list[no]) , con = conn)
            if len(kd_ini) < 8 :           
                cur.execute("insert into ods.kd (date ,date_no, no , k_value , d_value) values (%s,%s,%s,%s,%s)" , (insert_D ,date_no , all_list[no] , 50,50))
                
            else :
                
                process_no = pandas.read_sql("select * from (select date , no , close , highest , lowest from stock_daily where no = '{0}' and date in {1} union \
                                                             select date , no , close , highest , lowest from   otc_daily where no = '{0}' and date in {1}) main \
                                                      order by date ".format(all_list[no],date_list) , con = conn)
                #display(process_no)
                min_9 = process_no[process_no['lowest']>0]['lowest'].min()
                max_9 = process_no[process_no['highest']>0]['highest'].max()
                if min_9 == max_9:
                    RSV = 0
                else :
                    #display(process_no[process_no['close']>0])
                    #display(all_list[no])
                    try:
                        RSV = ((process_no[process_no['close']>0]['close'].iloc[-1]-min_9) / (max_9 - min_9))*100
                        last_kd = pandas.read_sql("select k_value , d_value from ods.kd where no = '{0}' order by date desc limit 1".format(all_list[no],date_list[-2]) , con = conn)
                        last_k = last_kd['k_value'][0]
                        last_d = last_kd['d_value'][0]            
                        
                        today_k = (last_k/3)*2 + (RSV/3)
                        today_d = (last_d/3)*2 + (today_k/3)
                        #print(RSV,today_k , today_d)
                
                        cur.execute("insert into ods.kd (date , date_no , no , k_value , d_value) values (%s,%s,%s,%s,%s)" , (insert_D , date_no , all_list[no] , today_k , today_d))
                        cur.execute("commit")        
        
                    except IndexError:
                        pass
        if len(all_list) == 0:
            print("【 KD 】{0} no need to inserted .".format(insert_D))
        else :
            print("【 KD 】{0} data inserted .".format(insert_D))

                

# 【 Technology Analyse 】- BIAS

In [ ]:
def BIAS(work_D,cur):
    for D in work_D:
        insert_D = '{:0>4}-{:0>2}-{:0>2}'.format(D.year,D.month,D.day)
        
        all_list = pandas.read_sql("select main.no \
                                      from (select date,no from stock_daily where date = '{0}'  union \
                                            select date , no from otc_daily where date = '{0}' ) main \
                                              join ods.analyze_avg avg \
                                                on avg.no = main.no \
                                               and avg.date = main.date \
                                              left join ods.bias b \
                                                on main.date = b.date \
                                               and main.no = b.no \
                                     where b.no is null".format(insert_D) , con = conn)['no'].tolist()
        
        if len(all_list) == 0:
            print("【 BIAS 】{0} no need to inserted .".format(insert_D))
        else :
            for i in range(len(all_list)):
                main = pandas.read_sql("select * from ods.analyze_avg where date = '{0}' and no = '{1}'".format(insert_D,all_list[i]) , con = conn)
                main['bias_5'] = round((main['close'][0] - main['sma5'][0])/main['sma5'][0],4)
                main['bias_20'] = round((main['close'][0] - main['sma20'][0])/main['sma20'][0],4)
                main = main[['date','date_no','no','bias_5','bias_20']]
                
                for _,data in main.iterrows():                                 
                    cur.execute("insert into ods.bias (date , date_no , no , bias_5,bias_20) values (%s,%s,%s,%s,%s)",data)
                cur.execute("commit")
                
                
            print("【 BIAS 】{0} inserted .".format(insert_D))

# 【 Report 】- FI Exponent Energy

In [ ]:
def FI_EXPONENT_ENERGY(work_D,cur):
    for D in work_D:
        
        # 串聯classify , institutional_investor , analyze_avg 取得投信及外資進出數量及成本
        insert_D = '{:0>4}-{:0>2}-{:0>2}'.format(D.year,D.month,D.day)
        
        FI = pandas.read_sql("select head.type , head.classify , head.date ,head.date_no , body.no , body.qty , body.amt \
                                from (select distinct date , date_no , type , classify from classify c , work_date d where d.date = '{0}') head \
                                left join (select main.date , d.date_no ,  main.no , c.type , c.classify , main.qty , main.amt \
                                        from (select fi.date , fi.no , fi.quantity as qty , fi.quantity * avg.price as AMT \
                                             from FOREIGN_INVESTOR fi \
                                             join STOCK_DAILY_AVG_PRICE avg \
                                               on fi.date = avg.date \
                                              and fi.no = avg.no ) main \
                                             join CLASSIFY c \
                                               on main.no = c.no \
                                             join work_date d \
                                               on main.date = d.date \
                                            where main.date = '{0}' ) body \
                                  on head.type = body.type \
                                 and head.classify = body.classify \
                                 and head.date = body.date ".format(insert_D) 
                        , con = conn)
        
        # 排除insert_D為非股市日
        if FI.shape[0] != 0 :
            #依照組別type, topic 加總金額及數量
            agg = FI[['type','classify','qty','amt']].groupby(['type','classify']).sum().reset_index()
            agg['date'] = FI['date'][0]
            agg['date_no'] = FI['date_no'][0]
            agg['investor'] = 'Foreign investor'
            agg[['amt','qty']].fillna(0)
            agg = agg[['date','date_no','type','classify','qty','amt','investor']]
            
            
            for _ , data in agg.iterrows():
                cur.execute("insert into ods.exponent_insti (date , date_no , type , topic , qty , amt , investor ) values (%s,%s,%s,%s,%s,%s,%s)",data)
            cur.execute('commit')
            
            print('【 Exponent Institutional 】{} Foreign Investor inserted. '.format(insert_D))
            agg = ''
        else:
            pass
        
        # 取得所有 type , topic
        
        all_type = pandas.read_sql("select distinct type from classify ",con = conn)
        
        
        for i_type in range(len(all_type)):   
        
            TYPE = all_type.iloc[i_type][0]
           
            all_topic = pandas.read_sql("select distinct classify from classify where type = '{}'".format(TYPE) , con = conn)
            for i_topic in range(len(all_topic)):
                
                TOPIC = all_topic.iloc[i_topic][0]
                
                main = pandas.read_sql("select * from ods.exponent_insti \
                                         where type = '{0}' \
                                           and topic = '{1}' \
                                           and investor = 'Foreign investor' \
                                           and date_no >= (select date_no from work_date where date = '{2}')-19 \
                                           and date_no <= (select date_no from work_date where date = '{2}') \
                                         order by date ".format(TYPE , TOPIC,insert_D) , con = conn)
                
                if len(main) == 20:
                    main['qty20'].iloc[-1] = main['qty'].sum()
                    main['amt20'].iloc[-1] = main['amt'].sum()
                    DATE = main['date'].iloc[-1]
                    #display(main.iloc[-1])
                    
                    cur.execute("update ods.exponent_insti set qty20 = '{0}' , amt20 = '{1}' \
                                  where date = '{2}' and type = '{3}' and topic = '{4}' and investor = 'Foreign investor'".format(main['qty20'].iloc[-1],main['amt20'].iloc[-1],DATE,TYPE,TOPIC))
                    
                    cur.execute("commit")
                else:
                    pass
        print('【 Exponent Institutional 】{0} qty20 & amt20 of Foreign Investor updated.'.format(insert_D))
                  
       

# 【 Report 】- IT Exponent Energy

In [6]:
def IT_EXPONENT_ENERGY(work_D,cur):
    for D in work_D:
        
        # 串聯classify , institutional_investor , analyze_avg 取得投信及外資進出數量及成本
        insert_D = '{:0>4}-{:0>2}-{:0>2}'.format(D.year,D.month,D.day)
        
        IT = pandas.read_sql("select head.type , head.classify , head.date ,head.date_no , body.no , body.qty , body.amt \
                                from (select distinct date , date_no , type , classify from classify c , work_date d where d.date = '{0}') head \
                                left join (select main.date , d.date_no ,  main.no , c.type , c.classify , main.qty , main.amt \
                                        from (select it.date , it.no , it.quantity as qty , it.quantity * avg.price as AMT \
                                             from INVESTMENT_TRUST it \
                                             join STOCK_DAILY_AVG_PRICE avg \
                                               on it.date = avg.date \
                                              and it.no = avg.no ) main \
                                             join CLASSIFY c \
                                               on main.no = c.no \
                                             join work_date d \
                                               on main.date = d.date \
                                            where main.date = '{0}' ) body \
                                  on head.type = body.type \
                                 and head.classify = body.classify \
                                 and head.date = body.date ".format(insert_D) 
                        , con = conn)
        
        # 排除insert_D為非股市日
        if IT.shape[0] != 0 :
            #依照組別type, topic 加總金額及數量
            agg = IT[['type','classify','qty','amt']].groupby(['type','classify']).sum().reset_index()
            agg['date'] = IT['date'][0]
            agg['date_no'] = IT['date_no'][0]
            agg['investor'] = 'Investment trust'
            agg = agg[['date','date_no','type','classify','qty','amt','investor']]
            #display(agg)
            
            for _ , data in agg.iterrows():
                cur.execute("insert into ods.exponent_insti (date , date_no , type , topic , qty , amt , investor ) values (%s,%s,%s,%s,%s,%s,%s)",data)
            cur.execute('commit')
            
            print('【 Exponent Institutional 】{} of Investment Trust inserted. '.format(insert_D))
            agg = ''
        else:
            pass
        # 取得所有 type , topic
        
        all_type = pandas.read_sql("select distinct type from classify ",con = conn)
        
        
        for i_type in range(len(all_type)):   
        
            TYPE = all_type.iloc[i_type][0]
           
            all_topic = pandas.read_sql("select distinct classify from classify where type = '{}'".format(TYPE) , con = conn)
            for i_topic in range(len(all_topic)):
                
                TOPIC = all_topic.iloc[i_topic][0]
                
                main = pandas.read_sql("select * from ods.exponent_insti \
                                         where type = '{0}' \
                                           and topic = '{1}' \
                                           and investor = 'Investment trust' \
                                           and date_no >= (select date_no from work_date where date = '{2}')-19 \
                                           and date_no <= (select date_no from work_date where date = '{2}') \
                                         order by date ".format(TYPE , TOPIC,insert_D) , con = conn)
                
                if len(main) == 20:
                    main['qty20'].iloc[-1] = main['qty'].sum()
                    main['amt20'].iloc[-1] = main['amt'].sum()
                    DATE = main['date'].iloc[-1]
                    #display(main.iloc[-1])
                    
                    cur.execute("update ods.exponent_insti set qty20 = '{0}' , amt20 = '{1}' \
                                  where date = '{2}' and type = '{3}' and topic = '{4}' and investor = 'Investment trust'".format(main['qty20'].iloc[-1],main['amt20'].iloc[-1],DATE,TYPE,TOPIC))
                    
                    cur.execute("commit")
                else:
                    pass
        print('【 Exponent Institutional 】{0} qty20 & amt20 of Investment trust updated.'.format(insert_D))
                

# 【 Distinct 】- Distinct Issued Amounts

In [ ]:
def DISTINCT_ISSUED_AMOUNTS(cur):
    main = pandas.read_sql("select date , no , count(*) from issued_amounts group by date , no having count(*) > 1" , con = conn)
    main = main[['date','no']]
    main['type'] = 'OTC'
    for _ , data in main.iterrows():
        cur.execute("delete from issued_amounts where date = %s and no = %s and type = %s",data)
    cur.execute("commit")
    
    print('DISTINCT_ISSUED_AMOUNTS is finished .')